<a href="https://colab.research.google.com/github/hadi-saad-75/NeuS/blob/main/npm3d_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip -q install --upgrade pip
!pip -q install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q uninstall -y numpy
!pip -q install -q "numpy==2.0.2"
!pip -q install -q --upgrade trimesh pyhocon icecream opencv-python tqdm scipy PyMCubes imageio imageio-ffmpeg scikit-image matplotlib

In [3]:
%cd /content
!git clone https://github.com/Totoro97/NeuS.git
%cd NeuS

/content
fatal: destination path 'NeuS' already exists and is not an empty directory.
/content/NeuS


In [4]:
!unzip -q /content/drive/MyDrive/neus_project/data/data_thin_structure.zip -d /content/NeuS/

replace /content/NeuS/thin_catbus/cameras_sphere.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
!unzip -q /content/drive/MyDrive/neus_project/pretrained/pretrained_thin_structure.zip -d /content/NeuS/exp/

replace /content/NeuS/exp/thin_catbus/womask/checkpoints/ckpt_300000.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!pwd
!ls

/content/NeuS
confs	       LICENSE		       public_data	 static
exp	       models		       README.md	 thin_catbus
exp_runner.py  preprocess_custom_data  requirements.txt  thin_cube


In [7]:
!ls -lh

total 64K
drwxr-xr-x 2 root root 4.0K Feb 26 17:07 confs
drwxr-xr-x 5 root root 4.0K Feb 26 16:43 exp
-rw-r--r-- 1 root root  18K Feb 26 16:17 exp_runner.py
-rw-r--r-- 1 root root 1.1K Feb 26 16:17 LICENSE
drwxr-xr-x 3 root root 4.0K Feb 26 16:43 models
drwxr-xr-x 5 root root 4.0K Feb 26 16:17 preprocess_custom_data
drwxr-xr-x 4 root root 4.0K Feb 26 16:53 public_data
-rw-r--r-- 1 root root 3.1K Feb 26 16:17 README.md
-rw-r--r-- 1 root root  140 Feb 26 16:17 requirements.txt
drwxr-xr-x 2 root root 4.0K Feb 26 16:17 static
drwxr-xr-x 4 root root 4.0K Aug 22  2021 thin_catbus
drwxr-xr-x 4 root root 4.0K Aug 22  2021 thin_cube


In [8]:
!ls thin_cube
!ls thin_catbus
!ls dragon

cameras_sphere.npz  image  mask
cameras_sphere.npz  image  mask
ls: cannot access 'dragon': No such file or directory


In [9]:
!ls exp

dragon	thin_catbus  thin_cube


In [10]:
!find exp -name "*.pth"

exp/thin_catbus/womask/checkpoints/ckpt_300000.pth
exp/thin_catbus/womask_sphere/womask/checkpoints/ckpt_300000.pth
exp/thin_catbus/womask_sphere/checkpoints/ckpt_300000.pth
exp/thin_cube/womask/checkpoints/ckpt_300000.pth
exp/thin_cube/womask_sphere/checkpoints/ckpt_300000.pth


In [11]:
!ls confs

thin_structure.conf  wmask.conf  womask.conf  womask_dtu_large_roi.conf


In [14]:
!ls public_data
!ls public_data/thin_cube

thin_catbus  thin_cube
cameras_sphere.npz  image  mask


In [16]:
!mv exp/thin_cube/womask exp/thin_cube/womask_sphere
!mv exp/thin_catbus/womask exp/thin_catbus/womask_sphere

mv: cannot move 'exp/thin_catbus/womask' to 'exp/thin_catbus/womask_sphere/womask': Directory not empty


In [17]:
!mv exp/thin_cube/womask_sphere/womask/* exp/thin_cube/womask_sphere/
!rm -rf exp/thin_cube/womask_sphere/womask

mv: cannot move 'exp/thin_cube/womask_sphere/womask/checkpoints' to 'exp/thin_cube/womask_sphere/checkpoints': Directory not empty


In [18]:
!find exp/thin_cube -maxdepth 3 -type d

exp/thin_cube
exp/thin_cube/womask_sphere
exp/thin_cube/womask_sphere/meshes
exp/thin_cube/womask_sphere/checkpoints
exp/thin_cube/wmask


In [19]:
!ls exp/thin_cube/womask_sphere/checkpoints

ckpt_300000.pth


In [20]:
# from /content/NeuS
!rm -rf exp/thin_cube/womask
!mv exp/thin_cube/womask_sphere exp/thin_cube/womask

In [23]:
!find exp/thin_cube -maxdepth 3 -type d
!ls exp/thin_cube/womask/checkpoints

exp/thin_cube
exp/thin_cube/womask
exp/thin_cube/womask/meshes
exp/thin_cube/womask/checkpoints
exp/thin_cube/wmask
ckpt_300000.pth


In [24]:
!test -d exp/thin_catbus/womask_sphere && (rm -rf exp/thin_catbus/womask; mv exp/thin_catbus/womask_sphere exp/thin_catbus/womask) || echo "thin_catbus OK"
!ls exp/thin_catbus/womask/checkpoints

ckpt_300000.pth


In [25]:
!grep -n "torch.load" -n exp_runner.py | head -n 20

210:        checkpoint = torch.load(os.path.join(self.base_exp_dir, 'checkpoints', checkpoint_name), map_location=self.device)


In [26]:
!perl -0777 -i -pe 's/torch\.load\(([^)]*?)map_location=self\.device\)/torch.load($1map_location=self.device, weights_only=False)/g' exp_runner.py

In [27]:
!grep -n "weights_only=False" -n exp_runner.py | head

In [28]:
!python exp_runner.py --mode validate_mesh --conf ./confs/womask.conf --case thin_cube --is_continue

2026-02-26 17:28:28.297685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772126908.318210   21634 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772126908.324527   21634 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772126908.340755   21634 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772126908.340785   21634 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772126908.340789   21634 computation_placer.cc:177] computation placer alr

In [29]:
!python - << 'PY'
from pyhocon import ConfigFactory
conf = ConfigFactory.parse_file("./confs/womask.conf")
print(conf["general"]["base_exp_dir"])
print(conf["dataset"]["data_dir"])
PY

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
./exp/CASE_NAME/womask_sphere
./public_data/CASE_NAME/


NameError: name 'PY' is not defined